# The Cancer Gene Atlas Cancer (TCGA) - Uveal Melanoma (UVM)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.5 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.3.1
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.2
pandas 0.22.0
clustergrammer_widget 1.9.0
seaborn 0.8.1
Fri May 11 16:35:47 2018 EDT

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Uveal Melanoma [UVM]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-V3-A9ZX,11073,UVM,TCGA,5019E7F1-DB46-4A07-9E7B-B6E5D1481B84,V3,56.0,NaN,417,Nationwide Children's Hospital,MX,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-V3-A9ZY,11074,UVM,TCGA,2112A42A-9B85-475A-A31B-4CE7751CDEB9,V3,54.0,NaN,417,Nationwide Children's Hospital,MX,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-V4-A9EW,11075,UVM,TCGA,6BE5176C-4980-4663-ADFF-D04F05737DA0,V4,44.0,NaN,417,Nationwide Children's Hospital,M0,...,21.0,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-V4-A9EV,11076,UVM,TCGA,DA6F0ED2-A6D5-4D87-823E-122EBD59AABA,V4,59.0,NaN,417,Nationwide Children's Hospital,M0,...,44.6,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-V4-A9EI,11077,UVM,TCGA,B533E555-90CF-4657-80C6-15B67DA8AD26,V4,78.0,NaN,417,Nationwide Children's Hospital,M0,...,27.0,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(80, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_UVM_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,babf8f75-0b6f-46ea-98fb-96ada0c8107c,081a619d-3c81-41e8-bcd5-a8a5dc06165f,ced1cf04-2acc-4979-a653-b63aafa14bdd,e5e6adc1-cf97-4fa0-9001-dff3bc724f37,a669894b-f97e-4b96-82dc-47548ed6d6b4,a2324550-9acb-42bd-b2b5-e6217b327e37,cf13a6fe-905d-4bab-b4ab-f08799010a73,647b66bc-e70c-42a4-bf16-466c886f0e6e,e192eb62-ef42-45ee-b27b-4d4234f72075,12c4897a-aab9-40e6-940e-fcf785d04ddc,...,590edf46-250f-45e5-9080-a95b0d85b47f,5189ff30-67eb-407e-b481-9426eca96908,50cc07f8-cd48-4d6c-9a0f-5c777cf32366,7fd637fc-6d56-4de1-84b4-dd0460c95edd,cfea40ed-d73b-4052-84a5-cf228f9af854,4b1d4b71-b78f-4c33-8dc9-7120894f5b9e,6b894228-ed7f-4907-ad92-a95cb3293a26,fceac077-5a7b-4bdf-a7fa-f937342c85d6,b863da66-25c4-43f7-bd2a-4737095c8fc9,dba72b72-e7ab-4971-8e01-3b130d235924
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,100.86900,126.92200,98.064800,82.386900,113.1900,103.587000,92.835000,106.811000,160.682000,128.705000,...,89.53690,79.9606,61.21460,76.078600,72.125800,129.273000,79.938600,103.907000,76.38230,99.9626
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,51.83470,9.69843,31.710600,28.001700,26.0191,22.182000,8.768970,18.756600,10.937600,41.640200,...,39.37950,57.7323,49.20710,25.002400,51.384300,34.581200,34.622800,23.621700,53.46660,33.1199
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,12.59570,11.91630,10.106000,27.101900,17.2191,30.727300,22.461800,20.335600,12.920700,32.204000,...,34.05260,17.3904,43.79560,48.064900,21.033600,30.975000,26.431400,22.604300,33.77290,23.3915
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,52.53400,33.16210,42.515800,64.578500,62.2617,32.077700,38.045400,53.818300,47.801700,54.059600,...,64.30320,85.8015,29.87360,22.691100,33.409400,75.468000,50.796800,37.515700,39.90730,56.8422
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.14239,0.00000,0.623015,0.143072,0.0000,0.230974,0.451513,0.144478,0.200825,0.633888,...,1.00937,1.3950,1.44956,0.710313,0.335533,0.434669,0.437537,0.243247,1.68611,0.0000


In [19]:
matrix.shape

(199169, 80)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,babf8f75-0b6f-46ea-98fb-96ada0c8107c,081a619d-3c81-41e8-bcd5-a8a5dc06165f,ced1cf04-2acc-4979-a653-b63aafa14bdd,e5e6adc1-cf97-4fa0-9001-dff3bc724f37,a669894b-f97e-4b96-82dc-47548ed6d6b4,a2324550-9acb-42bd-b2b5-e6217b327e37,cf13a6fe-905d-4bab-b4ab-f08799010a73,647b66bc-e70c-42a4-bf16-466c886f0e6e,e192eb62-ef42-45ee-b27b-4d4234f72075,12c4897a-aab9-40e6-940e-fcf785d04ddc,...,590edf46-250f-45e5-9080-a95b0d85b47f,5189ff30-67eb-407e-b481-9426eca96908,50cc07f8-cd48-4d6c-9a0f-5c777cf32366,7fd637fc-6d56-4de1-84b4-dd0460c95edd,cfea40ed-d73b-4052-84a5-cf228f9af854,4b1d4b71-b78f-4c33-8dc9-7120894f5b9e,6b894228-ed7f-4907-ad92-a95cb3293a26,fceac077-5a7b-4bdf-a7fa-f937342c85d6,b863da66-25c4-43f7-bd2a-4737095c8fc9,dba72b72-e7ab-4971-8e01-3b130d235924
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,100.86900,126.92200,98.064800,82.386900,113.1900,103.587000,92.835000,106.811000,160.682000,128.705000,...,89.53690,79.9606,61.21460,76.078600,72.125800,129.273000,79.938600,103.907000,76.38230,99.9626
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,51.83470,9.69843,31.710600,28.001700,26.0191,22.182000,8.768970,18.756600,10.937600,41.640200,...,39.37950,57.7323,49.20710,25.002400,51.384300,34.581200,34.622800,23.621700,53.46660,33.1199
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,12.59570,11.91630,10.106000,27.101900,17.2191,30.727300,22.461800,20.335600,12.920700,32.204000,...,34.05260,17.3904,43.79560,48.064900,21.033600,30.975000,26.431400,22.604300,33.77290,23.3915
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,52.53400,33.16210,42.515800,64.578500,62.2617,32.077700,38.045400,53.818300,47.801700,54.059600,...,64.30320,85.8015,29.87360,22.691100,33.409400,75.468000,50.796800,37.515700,39.90730,56.8422
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.14239,0.00000,0.623015,0.143072,0.0000,0.230974,0.451513,0.144478,0.200825,0.633888,...,1.00937,1.3950,1.44956,0.710313,0.335533,0.434669,0.437537,0.243247,1.68611,0.0000


In [22]:
matrix.shape

(79930, 80)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-WC-A87W-01,TCGA-V4-A9EA-01,TCGA-V4-A9EK-01,TCGA-YZ-A985-01,TCGA-WC-A885-01,TCGA-WC-A87U-01,TCGA-WC-A881-01,TCGA-WC-A882-01,TCGA-WC-A883-01,TCGA-YZ-A980-01,...,TCGA-V4-A9EX-01,TCGA-V4-A9EO-01,TCGA-VD-A8KO-01,TCGA-YZ-A983-01,TCGA-V4-A9EW-01,TCGA-V4-A9EQ-01,TCGA-V4-A9EV-01,TCGA-V4-A9EY-01,TCGA-V4-A9EU-01,TCGA-V4-A9EM-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,100.86900,126.92200,98.064800,82.386900,113.1900,103.587000,92.835000,106.811000,160.682000,128.705000,...,89.53690,79.9606,61.21460,76.078600,72.125800,129.273000,79.938600,103.907000,76.38230,99.9626
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,51.83470,9.69843,31.710600,28.001700,26.0191,22.182000,8.768970,18.756600,10.937600,41.640200,...,39.37950,57.7323,49.20710,25.002400,51.384300,34.581200,34.622800,23.621700,53.46660,33.1199
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,12.59570,11.91630,10.106000,27.101900,17.2191,30.727300,22.461800,20.335600,12.920700,32.204000,...,34.05260,17.3904,43.79560,48.064900,21.033600,30.975000,26.431400,22.604300,33.77290,23.3915
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,52.53400,33.16210,42.515800,64.578500,62.2617,32.077700,38.045400,53.818300,47.801700,54.059600,...,64.30320,85.8015,29.87360,22.691100,33.409400,75.468000,50.796800,37.515700,39.90730,56.8422
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,1.14239,0.00000,0.623015,0.143072,0.0000,0.230974,0.451513,0.144478,0.200825,0.633888,...,1.00937,1.3950,1.44956,0.710313,0.335533,0.434669,0.437537,0.243247,1.68611,0.0000


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-WC-A87W-01,TCGA-V4-A9EA-01,TCGA-V4-A9EK-01,TCGA-YZ-A985-01,TCGA-WC-A885-01,TCGA-WC-A87U-01,TCGA-WC-A881-01,TCGA-WC-A882-01,TCGA-WC-A883-01,TCGA-YZ-A980-01,...,TCGA-V4-A9EX-01,TCGA-V4-A9EO-01,TCGA-VD-A8KO-01,TCGA-YZ-A983-01,TCGA-V4-A9EW-01,TCGA-V4-A9EQ-01,TCGA-V4-A9EV-01,TCGA-V4-A9EY-01,TCGA-V4-A9EU-01,TCGA-V4-A9EM-01
ARF5,100.86900,126.92200,98.064800,82.386900,113.1900,103.587000,92.835000,106.811000,160.682000,128.705000,...,89.53690,79.9606,61.21460,76.078600,72.125800,129.273000,79.938600,103.907000,76.38230,99.9626
M6PR,51.83470,9.69843,31.710600,28.001700,26.0191,22.182000,8.768970,18.756600,10.937600,41.640200,...,39.37950,57.7323,49.20710,25.002400,51.384300,34.581200,34.622800,23.621700,53.46660,33.1199
ESRRA,12.59570,11.91630,10.106000,27.101900,17.2191,30.727300,22.461800,20.335600,12.920700,32.204000,...,34.05260,17.3904,43.79560,48.064900,21.033600,30.975000,26.431400,22.604300,33.77290,23.3915
FKBP4,52.53400,33.16210,42.515800,64.578500,62.2617,32.077700,38.045400,53.818300,47.801700,54.059600,...,64.30320,85.8015,29.87360,22.691100,33.409400,75.468000,50.796800,37.515700,39.90730,56.8422
CYP26B1,1.14239,0.00000,0.623015,0.143072,0.0000,0.230974,0.451513,0.144478,0.200825,0.633888,...,1.00937,1.3950,1.44956,0.710313,0.335533,0.434669,0.437537,0.243247,1.68611,0.0000


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_uvm_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining missing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-WC-A87W-01,TCGA-V4-A9EA-01,TCGA-V4-A9EK-01,TCGA-YZ-A985-01,TCGA-WC-A885-01,TCGA-WC-A87U-01,TCGA-WC-A881-01,TCGA-WC-A882-01,TCGA-WC-A883-01,TCGA-YZ-A980-01,...,TCGA-V4-A9EX-01,TCGA-V4-A9EO-01,TCGA-VD-A8KO-01,TCGA-YZ-A983-01,TCGA-V4-A9EW-01,TCGA-V4-A9EQ-01,TCGA-V4-A9EV-01,TCGA-V4-A9EY-01,TCGA-V4-A9EU-01,TCGA-V4-A9EM-01
A1BG,0.048325,0.062027,0.048325,0.048325,0.048325,0.048325,0.048325,0.048325,0.048325,0.048325,...,0.048325,0.048325,0.048325,0.048325,0.048325,0.048325,0.048325,0.048325,0.048325,0.048325
A1CF,0.015867,0.011006,0.011006,0.008205,0.003613,0.006432,0.007060,0.011006,0.015089,0.021407,...,0.013178,0.005292,0.002377,0.008775,0.004381,0.011006,0.009523,0.011006,0.025677,0.005291
A2M,878.125000,363.376000,139.125000,363.855000,128.843000,399.448000,129.052000,428.365000,281.041000,325.662000,...,974.653000,681.924000,274.773000,308.639000,934.098000,471.440000,1191.630000,178.863000,1342.400000,1738.020000
A2ML1,0.148874,0.022196,0.085695,0.021723,0.009373,0.027728,0.206085,0.040815,0.068742,0.222090,...,0.051658,0.046632,0.138154,0.064518,0.077983,0.024357,0.095923,0.008140,0.102466,0.024353
A3GALT2,0.019457,0.035392,0.035392,0.035392,0.035392,0.035392,0.020115,0.031014,0.016160,0.035392,...,0.035392,0.021093,0.035392,0.016579,0.035392,0.035392,0.078942,0.035392,0.045803,0.035392


In [34]:
normalized_matrix.shape

(18632, 80)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-WC-A87W-01,TCGA-V4-A9EA-01,TCGA-V4-A9EK-01,TCGA-YZ-A985-01,TCGA-WC-A885-01,TCGA-WC-A87U-01,TCGA-WC-A881-01,TCGA-WC-A882-01,TCGA-WC-A883-01,TCGA-YZ-A980-01,...,TCGA-V4-A9EX-01,TCGA-V4-A9EO-01,TCGA-VD-A8KO-01,TCGA-YZ-A983-01,TCGA-V4-A9EW-01,TCGA-V4-A9EQ-01,TCGA-V4-A9EV-01,TCGA-V4-A9EY-01,TCGA-V4-A9EU-01,TCGA-V4-A9EM-01
A1BG,-1.112541,-1.041859,-1.116406,-1.138237,-1.161416,-1.132703,-1.162650,-1.125367,-1.147228,-1.129614,...,-1.138480,-1.138405,-1.136157,-1.158006,-1.158557,-1.152429,-1.130093,-1.178441,-1.131887,-1.144305
A1CF,-1.596210,-1.792800,-1.758942,-1.908350,-2.287673,-2.008521,-1.998046,-1.767903,-1.652751,-1.483233,...,-1.702789,-2.098942,-2.444220,-1.898943,-2.201130,-1.794964,-1.835492,-1.820977,-1.406517,-2.104950
A2M,3.146843,2.725919,2.342826,2.738518,2.264472,2.784584,2.263943,2.822274,2.617369,2.698981,...,3.166198,3.011158,2.618644,2.647272,3.127663,2.836825,3.261876,2.389907,3.311822,3.411577
A2ML1,-0.623895,-1.488154,-0.867624,-1.485494,-1.873690,-1.373960,-0.532776,-1.198723,-0.994179,-0.467257,...,-1.109514,-1.153897,-0.679967,-1.032499,-0.950730,-1.449969,-0.832342,-1.952010,-0.805480,-1.441920
A3GALT2,-1.507631,-1.285537,-1.251679,-1.273511,-1.296689,-1.267976,-1.543304,-1.317981,-1.622970,-1.264887,...,-1.273753,-1.498444,-1.271431,-1.622626,-1.293830,-1.287702,-0.916958,-1.313714,-1.155166,-1.279578


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-WC-A87W-01,TCGA-V4-A9EA-01,TCGA-V4-A9EK-01,TCGA-YZ-A985-01,TCGA-WC-A885-01,TCGA-WC-A87U-01,TCGA-WC-A881-01,TCGA-WC-A882-01,TCGA-WC-A883-01,TCGA-YZ-A980-01,...,TCGA-V4-A9EX-01,TCGA-V4-A9EO-01,TCGA-VD-A8KO-01,TCGA-YZ-A983-01,TCGA-V4-A9EW-01,TCGA-V4-A9EQ-01,TCGA-V4-A9EV-01,TCGA-V4-A9EY-01,TCGA-V4-A9EU-01,TCGA-V4-A9EM-01
A1BG,-1.281546,-0.933294,-1.208284,-1.134300,-1.037380,-1.109222,-1.094349,-1.134105,-1.206689,-1.420444,...,-1.202015,-1.157875,-1.110660,-1.085539,-1.073387,-1.048548,-1.237550,-1.042940,-1.300042,-1.155242
A1CF,-1.707849,-1.789740,-1.866194,-2.011943,-2.246373,-1.997711,-1.945288,-1.853776,-1.683429,-1.685106,...,-1.730153,-2.172165,-2.671300,-1.829948,-2.087472,-1.744548,-1.897626,-1.768617,-1.543866,-2.172165
A2M,3.094957,2.808017,2.279649,2.803951,2.343860,2.824675,2.448871,2.837346,2.595729,2.558991,...,3.146505,2.912904,2.733492,2.800128,3.172014,2.765447,3.217228,2.508182,3.473579,3.360334
A2ML1,-0.835688,-1.464091,-0.973448,-1.516285,-1.852539,-1.353535,-0.420837,-1.215267,-1.065531,-0.961085,...,-1.173474,-1.175161,-0.655320,-0.957735,-0.876452,-1.378110,-0.978430,-1.909731,-1.001392,-1.461926
A3GALT2,-1.623095,-1.232839,-1.348781,-1.286324,-1.198252,-1.252566,-1.493628,-1.333451,-1.652230,-1.513358,...,-1.321148,-1.516904,-1.246337,-1.563008,-1.210486,-1.201095,-1.050661,-1.208050,-1.320614,-1.294498


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-WC-A87W-01,TCGA-V4-A9EA-01,TCGA-V4-A9EK-01,TCGA-YZ-A985-01,TCGA-WC-A885-01,TCGA-WC-A87U-01,TCGA-WC-A881-01,TCGA-WC-A882-01,TCGA-WC-A883-01,TCGA-YZ-A980-01,...,TCGA-V4-A9EX-01,TCGA-V4-A9EO-01,TCGA-VD-A8KO-01,TCGA-YZ-A983-01,TCGA-V4-A9EW-01,TCGA-V4-A9EQ-01,TCGA-V4-A9EV-01,TCGA-V4-A9EY-01,TCGA-V4-A9EU-01,TCGA-V4-A9EM-01
A1BG,-1.122248,1.723054,-0.523677,0.080787,0.872640,0.285679,0.407198,0.082381,-0.510650,-2.257077,...,-0.472462,-0.111824,0.273931,0.479176,0.578461,0.781398,-0.762793,0.827219,-1.273363,-0.090318
A1CF,0.666215,0.226738,-0.183560,-0.965738,-2.223833,-0.889360,-0.608027,-0.116916,0.797269,0.788271,...,0.546521,-1.825589,-4.504259,0.010961,-1.371076,0.469269,-0.352242,0.340097,1.546250,-1.825589
A2M,0.965469,0.006792,-1.758499,-0.006792,-1.543968,0.062447,-1.193123,0.104784,-0.702467,-0.825209,...,1.137693,0.357225,-0.242197,-0.019564,1.222917,-0.135435,1.373980,-0.994965,2.230457,1.852100
A2ML1,0.936700,-1.515332,0.399163,-1.718991,-3.031058,-1.083939,2.555453,-0.544418,0.039853,0.447403,...,-0.381343,-0.387923,1.640499,0.460475,0.777642,-1.179834,0.379722,-3.254222,0.290124,-1.506882
A3GALT2,-3.138730,0.358639,-0.680402,-0.120682,0.668598,0.181849,-1.978485,-0.543019,-3.399834,-2.155300,...,-0.432763,-2.187075,0.237676,-2.600245,0.558963,0.643118,1.991276,0.580791,-0.427980,-0.193929


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(18632, 80)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Filtered Matrix

In [52]:
filename = path+'tcga_uvm_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18136, 2)

# Save Gene List

In [56]:
filename = path+'tcga_uvm_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [57]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [58]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [59]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-RZ-AB0B-01,47,FEMALE,M1,NaN,T4b,Stage IV,Epithelioid Cell,Primary Solid Tumor,Choroid,NaN
TCGA-V3-A9ZX-01,56,MALE,MX,NX,T2b,Stage IIB,Uveal Melanoma; Spindle Cell,Primary Solid Tumor,Choroid,NaN
TCGA-V3-A9ZY-01,54,MALE,MX,NX,T3a,Stage IIB,Epithelioid Cell,Primary Solid Tumor,Choroid,NaN
TCGA-V4-A9E5-01,51,FEMALE,M0,N0,T4b,Stage IIIB,Uveal Melanoma; Epithelioid Cell,Primary Solid Tumor,Choroid,NaN
TCGA-V4-A9E7-01,76,MALE,M0,N0,T4a,Stage IIIA,Uveal Melanoma; Spindle Cell,Primary Solid Tumor,Choroid,NaN


In [60]:
attribute_list.shape

(80, 10)

# Save Attribute List

In [61]:
filename = path+'tcga_uvm_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [62]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [63]:
standard_matrix.head()

,TCGA-RZ-AB0B-01,TCGA-V3-A9ZX-01,TCGA-V3-A9ZY-01,TCGA-V4-A9E5-01,TCGA-V4-A9E7-01,TCGA-V4-A9E8-01,TCGA-V4-A9E9-01,TCGA-V4-A9EA-01,TCGA-V4-A9EC-01,TCGA-V4-A9ED-01,...,TCGA-WC-A885-01,TCGA-WC-A888-01,TCGA-WC-A88A-01,TCGA-WC-AA9A-01,TCGA-WC-AA9E-01,TCGA-YZ-A980-01,TCGA-YZ-A982-01,TCGA-YZ-A983-01,TCGA-YZ-A984-01,TCGA-YZ-A985-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.564696,-0.589700,0.560101,0.835514,0.335017,-0.189871,0.885636,0.910698,-0.539743,0.035012,...,0.735403,-0.239792,-0.739553,0.810532,0.134899,-0.864413,-0.489859,0.410039,-0.989219,0.084900
A1CF,0.972479,-0.228117,0.496791,0.596871,0.196523,-0.153115,0.772184,0.096668,-0.552949,0.121668,...,-0.777783,0.221550,0.421725,0.546785,0.872349,0.646954,-0.802753,-0.003202,-0.178155,-0.528049
A2M,0.246386,0.396445,-0.053503,-0.153441,-0.453365,-0.778104,-0.253395,-0.003522,-0.902986,0.371482,...,-0.753095,0.071383,-0.828073,-0.803067,0.346437,-0.478394,-0.703157,-0.053503,-0.628272,-0.028472
A2ML1,-0.325575,0.724708,-0.875101,-0.775244,0.874924,0.749716,-0.850147,-0.750238,-0.700303,0.249260,...,-0.950048,-0.450493,-0.925091,0.975044,0.849864,0.274234,-0.974975,0.374362,-0.400510,-0.800203
A3GALT2,-0.489005,-0.513903,-0.638836,0.761270,0.285795,-0.238953,0.886481,0.310850,0.836356,0.010916,...,0.560953,-0.339014,0.736243,-0.938504,0.060784,-0.688786,-0.463932,-0.763705,-0.039080,-0.064151


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [64]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [65]:
filename = path+'tcga_uvm_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [66]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [67]:
tertiary_matrix.head()

,TCGA-RZ-AB0B-01,TCGA-V3-A9ZX-01,TCGA-V3-A9ZY-01,TCGA-V4-A9E5-01,TCGA-V4-A9E7-01,TCGA-V4-A9E8-01,TCGA-V4-A9E9-01,TCGA-V4-A9EA-01,TCGA-V4-A9EC-01,TCGA-V4-A9ED-01,...,TCGA-WC-A885-01,TCGA-WC-A888-01,TCGA-WC-A88A-01,TCGA-WC-AA9A-01,TCGA-WC-AA9E-01,TCGA-YZ-A980-01,TCGA-YZ-A982-01,TCGA-YZ-A983-01,TCGA-YZ-A984-01,TCGA-YZ-A985-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,0
A1CF,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,...,-1,0,0,1,0,0,-1,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [68]:
filename = path+'tcga_uvm_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [69]:
name = 'tcga_uvm_gene_up_set'

In [70]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [71]:
name = 'tcga_uvm_gene_down_set'

In [72]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [73]:
name = 'tcga_uvm_attribute_up_set'

In [74]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [75]:
name = 'tcga_uvm_attribute_down_set'

In [76]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity Matrix

In [77]:
attribute_normalized_matrix = normalized_matrix.copy()

In [78]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [79]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [80]:
attribute_similarity_matix.head()

,"(Donor:TCGA-RZ-AB0B, Age:47.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V3-A9ZX, Age:56.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Uveal Melanoma; Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V3-A9ZY, Age:54.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V4-A9E5, Age:51.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIIB, Histology:Uveal Melanoma; Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V4-A9E7, Age:76.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Uveal Melanoma; Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V4-A9E8, Age:49.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Uveal Melanoma; Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V4-A9E9, Age:50.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Uveal Melanoma; Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V4-A9EA, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Uveal Melanoma; Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V4-A9EC, Age:75.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Uveal Melanoma; Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-V4-A9ED, Age:45.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Uveal Melanoma; Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)",...,"(Donor:TCGA-WC-A885, Age:60.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIA, Histology:Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-WC-A888, Age:76.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Uveal Melanoma; Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-WC-A88A, Age:75.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-WC-AA9A, Age:70.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIA, Histology:Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-WC-AA9E, Age:60.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIA, Histology:Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-YZ-A980, Age:75.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-YZ-A982, Age:79.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIIB, Histology:Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-YZ-A983, Age:51.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-YZ-A984, Age:50.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Uveal Melanoma; Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)","(Donor:TCGA-YZ-A985, Age:41.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-RZ-AB0B, Age:47.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Epithelioid Cell, Tissue:Choroid, Tumor:nan, Gradenan)",1.000000,0.293054,-0.209514,-0.194650,0.078547,-0.033509,-0.296674,-0.226046,-0.188568,0.002621,...,-0.233325,0.107998,-0.018974,0.073662,0.062635,0.130638,0.006605,-0.008751,-0.029138,-0.075573
"(Donor:TCGA-V3-A9ZX, Age:56.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Uveal Melanoma; Spindle Cell, Tissue:Choroid, Tumor:nan, Gradenan)",0.293054,1.000000,-0.172284,-0.120745,0.091711,0.040931,-0.242973,-0.165243,-0.208045,0.051211,...,-0.1708

# Save Attribute Similarity Matrix

In [81]:
filename = path+'tcga_uvm_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [82]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

clustergrammer_widget(network='{"mat": [[1.0, 0.2930538816097805, -0.20951383497165388, -0.194649835292529, 0.…

# Create Gene Similarity Matrix

In [83]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [84]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.002766,0.073633,0.055392,0.040590,-0.147036,0.013055,-0.194123,-0.116983,0.063628,...,0.042109,-0.030084,0.111274,0.268779,0.017761,0.231959,0.024890,-0.001378,-0.104581,0.004705
A1CF,-0.002766,1.000000,0.231462,-0.036165,-0.046765,0.178373,0.060168,-0.142934,0.137980,-0.102051,...,0.010584,0.018765,0.017504,-0.060698,-0.273895,0.146868,-0.018949,-0.035462,0.039480,-0.020982
A2M,0.073633,0.231462,1.000000,0.033533,0.044150,0.028599,-0.007360,-0.024620,0.067744,-0.130161,...,0.278478,0.144766,0.120781,0.315649,-0.084976,0.061419,0.261658,-0.210151,-0.071130,0.343600
A2ML1,0.055392,-0.036165,0.033533,1.000000,-0.140276,-0.099461,-0.088895,-0.334384,-0.229827,-0.146630,...,0.055836,-0.205843,0.099963,0.215306,-0.246671,0.158073,0.139177,-0.186313,-0.293813,0.071383
A3GALT2,0.040590,-0.046765,0.044150,-0.140276,1.000000,0.185222,-0.077626,0.136421,0.259117,0.059490,...,-0.262420,0.104436,0.060497,0.013239,0.040340,-0.083434,-0.138750,0.219951,0.149032,-0.051707


# Save Gene Similarity Matrix

In [85]:
filename = path+'tcga_uvm_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [86]:
name = 'tcga_uvm_gene_attribute_edge_list'

In [87]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  80 Out of 80   

 The number of statisticaly relevent gene-attribute associations is: 73121
